In [10]:
import pandas as pd
import numpy as np
from exerpy.analyses import _load_json
from exerpy import ExergyAnalysis

In [11]:
def diff_between_simulators(testcase):
    simulator_results = []
    for path in testcase.values():
        contents = _load_json(path)
        if "settings" not in contents:
            contents["settings"] = {}

        simulator_results += [
            ExergyAnalysis.from_json(path, **contents["settings"])
        ]

    sim1 = simulator_results[0]
    sim2 = simulator_results[1]

    columns = ["m", "p", "T"]
    if sim1.chemExLib is not None:
        columns.append("e_CH")
    if sim1.split_physical_exergy:
        columns.append("e_M")
        columns.append("e_T")
    else:
        columns.append("e_PH")

    df_sim1 = pd.DataFrame.from_dict(
        sim1._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")
    df_sim2 = pd.DataFrame.from_dict(
        sim2._connection_data, orient="index"
    ).sort_index()[columns].dropna(how="all")

    overlapping_index = list(
        set(df_sim1.index.tolist()) & set(df_sim2.index.tolist())
    )
    df_sim1 = df_sim1.loc[overlapping_index].round(6)
    df_sim2 = df_sim2.loc[overlapping_index].round(6)

    # inf means that sim2 has 0 value, comparison does not make sense there
    # and sometimes there seem to be NaN values in the dataframes, those are
    # removed as well
    return (
        (df_sim1 - df_sim2) / df_sim2
    ).abs().replace(np.inf, 0).fillna(0)

In [12]:
testcase = {
    "ebsilon": "ccpp_ebs.json",
    "tespy": "ccpp_tespy.json"
}
diff_between_simulators(testcase)


,m,p,T,e_CH,e_PH
11,0.000051,0.000000e+00,9.288467e-05,0.000000,5.951819e-05
7,0.000162,4.509804e-10,3.838040e-08,0.000169,5.199032e-04
14,0.000665,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
17,0.000601,0.000000e+00,0.000000e+00,0.000000,1.303753e-08
12,0.000601,0.000000e+00,9.288467e-05,0.000000,5.951819e-05
3,0.000303,0.000000e+00,0.000000e+00,0.000000,1.047781e-02
22,0.000317,1.706295e-10,1.882696e-09,0.000000,2.027239e-10
13,0.000601,0.000000e+00,0.000000e+00,0.000000,3.205616e-05
9,0.000317,0.000000e+00,8.707701e-05,0.000000,6.942330e-05
15,0.000665,0.000000e+00,4.135862e-06,0.000000,1.904954e-04


In [13]:
testcase = {
    "tespy": "ccpp_tespy.json",
    "aspen": "ccpp_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_CH,e_PH
11,0.000076,0.0,1.367758e-04,0.000000,8.830437e-05
7,0.000710,0.0,7.037865e-04,0.001740,2.609650e-03
14,0.000258,0.0,0.000000e+00,0.000000,0.000000e+00
17,0.000307,0.0,2.123432e-07,0.000000,4.365327e-06
12,0.000307,0.0,1.367758e-04,0.000000,8.830437e-05
3,0.001741,0.0,0.000000e+00,0.000000,1.034113e-02
22,0.000099,0.0,6.024663e-06,0.000000,2.476646e-05
13,0.000307,0.0,0.000000e+00,0.000000,4.785170e-05
9,0.000099,0.0,1.282123e-04,0.000000,1.028809e-04
15,0.000258,0.0,4.135879e-06,0.000000,1.912064e-04


In [14]:
testcase = {
    "ebsilon": "ccpp_ebs.json",
    "aspen": "ccpp_aspen.json"
}
diff_between_simulators(testcase)

,m,p,T,e_CH,e_PH
11,0.000025,0.000000e+00,4.387841e-05,0.000000e+00,2.878092e-05
7,0.000871,4.509804e-10,7.037481e-04,1.909649e-03,3.130910e-03
14,0.000924,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
17,0.000908,0.000000e+00,2.123432e-07,0.000000e+00,4.352289e-06
12,0.000908,0.000000e+00,4.387841e-05,0.000000e+00,2.878092e-05
3,0.002044,0.000000e+00,0.000000e+00,0.000000e+00,2.832279e-05
22,0.000416,1.706295e-10,6.022781e-06,0.000000e+00,2.476626e-05
13,0.000908,0.000000e+00,0.000000e+00,0.000000e+00,1.579400e-05
9,0.000416,0.000000e+00,4.112415e-05,0.000000e+00,3.345047e-05
15,0.000924,0.000000e+00,0.000000e+00,0.000000e+00,6.745670e-07


In [15]:
# Load the CSV files
tespy_results = pd.read_csv("ccpp_components_tespy.csv", index_col=0)
ebsilon_results = pd.read_csv("ccpp_components_ebsilon.csv", index_col=0)

# Step 1: Normalize the component names in Tespy
# Group "drum" and "drum pump" into "EVA"
tespy_results.loc[tespy_results["Component"].isin(["drum", "drum pump"]), "Component"] = "EVA"

# Group "dea steam inlet valve" into "DEA"
tespy_results.loc[tespy_results["Component"] == "dea steam inlet valve", "Component"] = "DEA"

# Group "MIX" into "DEA"
ebsilon_results.loc[ebsilon_results["Component"] == "MIX", "Component"] = "DEA"

# Aggregate the values by summing them up
tespy_results = tespy_results.groupby("Component", as_index=False).sum(numeric_only=True)
ebsilon_results = ebsilon_results.groupby("Component", as_index=False).sum(numeric_only=True)

# Step 2: Merge DataFrames on Component name
merged_df = pd.merge(ebsilon_results, tespy_results, on="Component", suffixes=("_Ebsilon", "_Tespy"))

# Step 3: Compute Differences
columns_to_compare = ["E_F [kW]", "E_P [kW]", "E_D [kW]", "E_L [kW]", "ε [%]", "y [%]", "y* [%]"]
for col in columns_to_compare:
    merged_df[f"Diff_{col}"] = merged_df[f"{col}_Tespy"] - merged_df[f"{col}_Ebsilon"]

# Step 4: Select only the relevant columns
diff_columns = [col for col in merged_df.columns if col.startswith("Diff_E_D") or col.endswith("E_D [kW]_Ebsilon") or col.endswith("E_D [kW]_Tespy")]
diff_df = merged_df[["Component"] + diff_columns].copy()
diff_df["Relative error [%]"] = diff_df["Diff_E_D [kW]"] / diff_df["E_D [kW]_Tespy"] * 100

# Display the filtered DataFrame
diff_df

,Component,E_D [kW]_Ebsilon,E_D [kW]_Tespy,Diff_E_D [kW],Relative error [%]
0,CC,195766.602870,195534.635572,-231.967298,-0.118632
1,COMP,10945.496471,10953.651539,8.155068,0.074451
2,COND,3717.826764,3715.714434,-2.112330,-0.056849
3,DEA,3184.745810,3183.009791,-1.736019,-0.054540
4,ECO,1475.374949,1474.821208,-0.553741,-0.037546
5,EVA,11648.296333,11650.581602,2.285268,0.019615
6,GT,15969.551601,15984.355294,14.803693,0.092614
7,HC,20.937228,0.000000,-20.937228,-inf
8,PUMP1,7.087864,7.083607,-0.004257,-0.060102
9,PUMP2,57.850131,57.831740,-0.018390,-0.031800
